# UNIÓN DE TABLAS - BUSQUEDA MANUAL
El objetivo de este archivo es continuar con la busqueda del nombre INCI que es la que aparecerá en los productos. 
Partimos de la base de que tenemos una lista de disruptores endocrinos, pero dichos disruptores pueden ser sustancias o ingredientes utilizadas para la elaboración de:
- productos cosméticos.
- pesticidas
- fungicidas
- productos de limpieza
- y un largo etc...

El cruce con la base de datos de sustancias activas de pesticidas no ha sido suficiente.

Para nuestra búsqueda de nombres INCI vamos a realizar los siguientes pasos:

1. Unión nuevamente de la tabla de disruptores limpia de pesticidas **"disruptores_clean"** con la tabla que tienen los nombres de las etiquetas **"disruptores_etiqueta"**, para poder filtrar las que faltan por rellenar.
2. Busqueda manual de nombres INCI en varias fuentes que no disponen ni de API ni de archivos para hacer merges
3. Tratamiento y exportación de listado final para tratarlo con NLP

La primera unión -> **disruptores_clean** y **disruptores_etiqueta**: 

* Se harán por las claves: CAS no. y EC no.
* Objetivo: Unir para quitar de diruptores_clean los que ya tienen etiqueta y con ello hacer búsqueda manual.


## 1. Importamos librerias

In [1]:
import pandas as pd
import numpy as np

## 2. Lectura de datos de "disrupores_clean" y "disruptores_etiqueta"

In [2]:
# leemos datasets de disrupores_clean
disruptores_clean = pd.read_parquet("../../data/processed/notebooks/disruptores_clean.parquet")
disruptores_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name_Edlist_Echa  292 non-null    object 
 1   Fuente_original   292 non-null    object 
 2   Appears on lists  236 non-null    object 
 3   Health effects    236 non-null    float64
 4   CAS Number        279 non-null    object 
 5   EC Number         247 non-null    object 
dtypes: float64(1), object(5)
memory usage: 13.8+ KB


In [3]:
# leemos datasets de disrupores_etiqueta
disruptores_etiqueta = pd.read_parquet("../../data/processed/notebooks/disruptores_etiqueta.parquet")
disruptores_etiqueta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name_Edlist_Echa      36 non-null     object 
 1   Name_Chemical_Cosing  36 non-null     object 
 2   Fuente_original       36 non-null     object 
 3   Anexo_cosIng          36 non-null     object 
 4   Appears on lists      26 non-null     object 
 5   Health effects        26 non-null     float64
 6   CAS Number            36 non-null     object 
 7   EC Number             36 non-null     object 
 8   nombre_etiqueta       36 non-null     object 
dtypes: float64(1), object(8)
memory usage: 2.7+ KB


## 2. Disruptores sin nombre INCI

In [4]:
# consultamos duplicados
duplicados = disruptores_clean[disruptores_clean.duplicated(subset='Name_Edlist_Echa', keep=False)].sort_values(by=['CAS Number']) 
duplicados.head()

,Name_Edlist_Echa,Fuente_original,Appears on lists,Health effects,CAS Number,EC Number
0,2-tert-butyl-4-methoxyphenol (BHA),Edlist_2,List II,1.0,121-00-6,204-442-7
1,2-tert-butyl-4-methoxyphenol (BHA),Edlist_2,List II,1.0,121-00-6,246-563-8
2,"Lithium salts (lithium chloride, lithium carbo...",Edlist_3,List III,1.0,1310-65-2,209-062-5
3,"Lithium salts (lithium chloride, lithium carbo...",Edlist_3,List III,1.0,1310-65-2,215-183-4
4,"Lithium salts (lithium chloride, lithium carbo...",Edlist_3,List III,1.0,1310-65-2,231-212-3


No podemos eliminar duplicados por la combinación que hay entre CAS Number y EC Number. Es mejor manterner para tener más posibilidades de encontrar el nombre de la etiqueta en otra base de datos.

In [5]:
# Hacemos merge entre disrupores_clean y disrupores_etiqueta
disruptores_provisional = disruptores_clean.merge(disruptores_etiqueta[[ 'CAS Number', 'EC Number','Name_Chemical_Cosing', 'nombre_etiqueta',  'Anexo_cosIng']], on=['CAS Number', 'EC Number'], how='left' )

disruptores_provisional.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name_Edlist_Echa      298 non-null    object 
 1   Fuente_original       298 non-null    object 
 2   Appears on lists      241 non-null    object 
 3   Health effects        241 non-null    float64
 4   CAS Number            285 non-null    object 
 5   EC Number             253 non-null    object 
 6   Name_Chemical_Cosing  36 non-null     object 
 7   nombre_etiqueta       36 non-null     object 
 8   Anexo_cosIng          36 non-null     object 
dtypes: float64(1), object(8)
memory usage: 21.1+ KB


In [6]:
disruptores_provisional['nombre_etiqueta'].nunique(dropna=True)

36

In [7]:
disruptores_provisional[['Name_Edlist_Echa']].nunique(dropna=True)

Name_Edlist_Echa    257
dtype: int64

Como vemos la variable 'Name_Edlist_Echa' que venía sin nulos antes de la unión tiene 257 registros únicos de los cuales hay 36 registros únicos del 'nombre_etiqueta'.
Por lo que después del filtrado deberían quedar 257-36 = 221 registros sin nombre de etiqueta (eliminando duplicados obviamente)

In [8]:
# consultamos duplicados sin tener en cuenta el primer registro, es decir los que vamos a eliminar
duplicados = disruptores_provisional[disruptores_provisional.duplicated(subset=['Name_Edlist_Echa' ], keep='first')].sort_values(by=['nombre_etiqueta', 'Name_Chemical_Cosing']) 
duplicados.count()

Name_Edlist_Echa        41
Fuente_original         41
Appears on lists        38
Health effects          38
CAS Number              40
EC Number               38
Name_Chemical_Cosing     6
nombre_etiqueta          6
Anexo_cosIng             6
dtype: int64

298 registros totales - 257 únicos = 41 duplicados. 

In [9]:
disruptores_provisional = disruptores_provisional.drop_duplicates(subset=['Name_Edlist_Echa'], keep='first')

In [10]:
disruptores_provisional.info()

<class 'pandas.core.frame.DataFrame'>
Index: 257 entries, 0 to 297
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name_Edlist_Echa      257 non-null    object 
 1   Fuente_original       257 non-null    object 
 2   Appears on lists      203 non-null    object 
 3   Health effects        203 non-null    float64
 4   CAS Number            245 non-null    object 
 5   EC Number             215 non-null    object 
 6   Name_Chemical_Cosing  30 non-null     object 
 7   nombre_etiqueta       30 non-null     object 
 8   Anexo_cosIng          30 non-null     object 
dtypes: float64(1), object(8)
memory usage: 20.1+ KB


Ahora en vez de eliminar 36 registros se van a eliminar 30, por tanto obtendremos un dataframe de 257-30 = 227

In [11]:
disruptores_sin_etiqueta = disruptores_provisional[(disruptores_provisional['nombre_etiqueta'].isnull())]
disruptores_sin_etiqueta.info()

<class 'pandas.core.frame.DataFrame'>
Index: 227 entries, 0 to 297
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name_Edlist_Echa      227 non-null    object 
 1   Fuente_original       227 non-null    object 
 2   Appears on lists      182 non-null    object 
 3   Health effects        182 non-null    float64
 4   CAS Number            215 non-null    object 
 5   EC Number             185 non-null    object 
 6   Name_Chemical_Cosing  0 non-null      object 
 7   nombre_etiqueta       0 non-null      object 
 8   Anexo_cosIng          0 non-null      object 
dtypes: float64(1), object(8)
memory usage: 17.7+ KB


In [12]:
# Eliminamos variables que ya no tienen registros.
disruptores_sin_etiqueta = disruptores_sin_etiqueta[['CAS Number', 'EC Number', 'Name_Edlist_Echa', 'Fuente_original', 'Appears on lists','Health effects' ]]

In [13]:
disruptores_sin_etiqueta.head()

,CAS Number,EC Number,Name_Edlist_Echa,Fuente_original,Appears on lists,Health effects
0,121-00-6,204-442-7,2-tert-butyl-4-methoxyphenol (BHA),Edlist_2,List II,1.0
2,1310-65-2,209-062-5,"Lithium salts (lithium chloride, lithium carbo...",Edlist_3,List III,1.0
5,1506-02-1,216-133-4,"1-(5,6,7,8-tetrahydro-3,5,5,6,8,8-hexamethyl-2...",Edlist_2,List II,0.0
7,540-97-6,208-762-8,Cyclomethicone,Edlist_2,List II,1.0
22,100532-36-3,None,4-(5-methylhexyl)phenol,Edlist_1,List I,0.0


In [14]:
disruptores_sin_etiqueta['Name_Edlist_Echa'].nunique(dropna=True)

227

El objetivo es encontrar el nombre INCI a los 227 registros, pero sabemos que pueden haber sustancias que no se corresponden a productos cosméticos.

**EXPORTAMOS PARA SU TRATAMIENTO MANUAL**

In [15]:
# EXCEL
disruptores_sin_etiqueta.to_excel("../../data/processed/notebooks/disruptores_sin_etiqueta.xlsx", index=False)

## 3. Recuperación de listados con tratemiento manual con nombre INCI

Se han realizado tratamientos manuales de los dos archivos, 

In [16]:
# leemos datasets de disrupores_clean
inci_manual = pd.read_excel("../../data/processed/notebooks/consultas_manuales/disruptores_sin_etiqueta_manual.xlsx")

In [17]:
# leemos datasets de disrupores_clean
inci = pd.read_excel("../../data/processed/notebooks/consultas_manuales/disruptores_etiqueta_manual.xlsx")


## 4. Tratamiento de "inci_manual"

Este dataframe tiene más variables y más columnas de las iniciales por los siguientes motivos:
1. Se han añadido variables como:
* "nombre_etiqueta" : está es la que está en el otro dataframe en el que si se consiguió el nombre con el merge pero esta variable se ha rellenado de forma manual
* "fuente_manual", "productos_cosmeticos", "uso", "imagen_nombre", "imagen_cantidad" "texto", "repetido".
2. La variable "imagen_nombre" hace referencia al nombre de las capturas de pantalla de etiqueta de dicho disruptor, para una evaluación posterior en ocr y nlp.
3. La variable "imagen_cantidad" hace referencia a la cantidad de imágenes que se han hecho para esa sustancia
4. La variable "texto" es el texto escrito de una de esas imagenes.
5. Se han añadido filas debido a que un producto puede tomar más nombres, por lo que se ha duplicado toda la información, salvo el "nombre_etiqueta" que contiene los distintos alias. SEría como un "explode" manual.

Vamos a obtener el listado disruptores con nombre INCI, para ello:

1. Filtramos por `nombre_etiqueta`
2. Quitamos los que la variable `repetidos` ==1 identificados en la búsqueda manual porque tenían puesta la formulación distinta pero se trataba de la misma sustancia. Por ejemplo:
    * Nonylphenol, ethoxylated (7-EO)
    * 23-(nonylphenoxy)-3,6,9,12,15,18,21-heptaoxatricosan-1-ol
    * Poly (oxy-1,2-ethanediyl), alpha -(nonylphenyl)-omega-hydroxy-, branched   --> es la forma generica para el mismo y otros terminados en otros números.
3. Filtramos por la variable `productos_cosmeticos` == 1, y los que tienen valor "0" o nulo es porque o no lo son o se desconoce que se use en productos cosméticos.

In [18]:
inci_manual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283 entries, 0 to 282
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name_Edlist_Echa      283 non-null    object 
 1   Fuente_original       283 non-null    object 
 2   Anexo_cosIng          26 non-null     object 
 3   Appears_on_lists      234 non-null    object 
 4   Health_effects        234 non-null    float64
 5   CAS Number            271 non-null    object 
 6   EC Number             226 non-null    object 
 7   nombre_etiqueta       163 non-null    object 
 8   fuente_manual         163 non-null    object 
 9   repetido              15 non-null     float64
 10  productos_cosmeticos  145 non-null    float64
 11  uso                   136 non-null    object 
 12  imagen_cantidad       126 non-null    float64
 13  imagen_nombre         35 non-null     object 
 14  texto                 35 non-null     object 
dtypes: float64(4), object(1

In [19]:
# Filtramos por nombre manual
inci_manual = inci_manual[inci_manual['nombre_etiqueta'].notnull()]
# Quitamos los repetidos
inci_manual = inci_manual[inci_manual['repetido'].isnull()]


In [20]:
# Filtramos 'productos_cosmeticos' que toman valor 0
consulta= inci_manual[inci_manual['productos_cosmeticos']==0]
consulta[['productos_cosmeticos', "uso"]]

,productos_cosmeticos,uso
6,0.0,"Materia prima para detergentes, pesticidas, an..."
14,0.0,fungicida plantas
23,0.0,"fabricar plásticos, fertilizantes y hormigón"
24,0.0,"Retardante de llama en epoxi, poliuretano, plá..."
31,0.0,Materia prima para la fabricación de detergent...
32,0.0,Materia prima para la fabricación de detergent...
34,0.0,Plastificante ampliamente presente en el medio...
35,0.0,"Fungicida fenilpirrol no sistémico, aplicado e..."
44,0.0,Utilizado en la producción de resinas fenol-fo...
45,0.0,Herbicida destinado principalmente a la aplica...


In [21]:
inci_manual= inci_manual[inci_manual['productos_cosmeticos']==1]

In [22]:
inci_manual.info()

<class 'pandas.core.frame.DataFrame'>
Index: 104 entries, 0 to 270
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name_Edlist_Echa      104 non-null    object 
 1   Fuente_original       104 non-null    object 
 2   Anexo_cosIng          1 non-null      object 
 3   Appears_on_lists      90 non-null     object 
 4   Health_effects        90 non-null     float64
 5   CAS Number            104 non-null    object 
 6   EC Number             86 non-null     object 
 7   nombre_etiqueta       104 non-null    object 
 8   fuente_manual         104 non-null    object 
 9   repetido              0 non-null      float64
 10  productos_cosmeticos  104 non-null    float64
 11  uso                   103 non-null    object 
 12  imagen_cantidad       104 non-null    float64
 13  imagen_nombre         35 non-null     object 
 14  texto                 35 non-null     object 
dtypes: float64(4), object(11)
me

In [23]:
# Quitamos la variable 'repetido' porque ya no tiene sentido
inci_manual = inci_manual.drop(columns=['repetido'], errors='ignore')

Vamos a pasar las variables float a numérico pero antes vamos a convertir sus nulos en '0'

In [24]:
cols = ["Health_effects","productos_cosmeticos", "imagen_cantidad"]

inci_manual[cols] = inci_manual[cols].fillna(0).astype(int)

Ahora vamos a consultar en cuantos de los 227 registros de salida hemos coseguido su nombre INCI.

Aunque hayamos encontrado más de uno para alguna sustancia, lo que queremos saber de esos 227 cuales se han rellenado.

In [25]:
inci_manual['nombre_etiqueta'].nunique(dropna=True)

104

In [26]:
inci_manual['Name_Edlist_Echa'].nunique(dropna=True)

47

Esto quiere decir que de los 227 hemos conseguido rellenar 47 y que esas 47 sustancias tienen por termino medio 2 nombres, porque tenemos hasta 104 nombres de etiqueta

## 5. Tratamiento de "inci" - dataframe inicial: disruptores_etiqueta

Este dataframe tiene más variables y más columnas de las iniciales por los siguientes motivos:
1. Se han añadido variables como: 
* "uso","productos_cosmeticos", , "imagen_nombre", "imagen_cantidad", "texto".

2. Al igual que con el otro dataframe se han añadido filas debido a que un producto puede tomar más nombres.

In [27]:
# Consultamos la información de inci.
inci.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name_Edlist_Echa      52 non-null     object 
 1   Name_Chemical_Cosing  52 non-null     object 
 2   Fuente_original       52 non-null     object 
 3   Anexo_cosIng          52 non-null     object 
 4   Appears_on_lists      37 non-null     object 
 5   Health_effects        43 non-null     float64
 6   CAS Number            52 non-null     object 
 7   EC Number             52 non-null     object 
 8   nombre_etiqueta       52 non-null     object 
 9   uso                   40 non-null     object 
 10  productos_cosmeticos  52 non-null     int64  
 11  imagen_nombre         28 non-null     object 
 12  imagen_cantidad       50 non-null     float64
 13  texto                 28 non-null     object 
dtypes: float64(2), int64(1), object(11)
memory usage: 5.8+ KB


Vamos a pasar la variable float a numérico pero antes vamos a convertir sus nulos en '0'

In [28]:
inci["Health_effects"] = inci["Health_effects"].fillna(0).astype(int)
inci["Health_effects"] = (inci["Health_effects"] == 1).astype(int)

## 6. LISTA DEFINITIVA DE DISRUPTORES EN PRODUCTOS COSMÉTICOS

In [29]:
# Concatenamos los DataFrames
disruptores_final = pd.concat([inci, inci_manual], ignore_index=True)
disruptores_final.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name_Edlist_Echa      156 non-null    object 
 1   Name_Chemical_Cosing  52 non-null     object 
 2   Fuente_original       156 non-null    object 
 3   Anexo_cosIng          53 non-null     object 
 4   Appears_on_lists      127 non-null    object 
 5   Health_effects        156 non-null    int64  
 6   CAS Number            156 non-null    object 
 7   EC Number             138 non-null    object 
 8   nombre_etiqueta       156 non-null    object 
 9   uso                   143 non-null    object 
 10  productos_cosmeticos  156 non-null    int64  
 11  imagen_nombre         63 non-null     object 
 12  imagen_cantidad       154 non-null    float64
 13  texto                 63 non-null     object 
 14  fuente_manual         104 non-null    object 
dtypes: float64(1), int64(2)

In [30]:
# es importante que estas columnas sean de tipo string
cols = ['CAS Number', 'EC Number']
disruptores_final[cols] = (
    disruptores_final[cols]
      .apply(lambda col: col.astype('string').str.strip())
)

In [31]:
# las variables EC Number y Anexo_cosIng tienen valores nulos que vamos a rellenar
disruptores_final["EC Number"] = disruptores_final["EC Number"].fillna("")
disruptores_final["Anexo_cosIng"] = disruptores_final["Anexo_cosIng"].fillna("sin info")

Vamos a comprobar si el número de imagenes físicas que tenemos coincide con la suma de la unión de ambos dataframes: 
Si coincide

In [32]:
total_imagenes = disruptores_final["imagen_cantidad"].sum()
print(total_imagenes)

170.0


In [33]:
# Filtramos filas con varios nombres que no se dividió a mano, porque no había foto
disruptores_final[disruptores_final['nombre_etiqueta'].str.contains(r'[/;]', regex= True,na=False)]

,Name_Edlist_Echa,Name_Chemical_Cosing,Fuente_original,Anexo_cosIng,Appears_on_lists,Health_effects,CAS Number,EC Number,nombre_etiqueta,uso,productos_cosmeticos,imagen_nombre,imagen_cantidad,texto,fuente_manual
67,tert-butyl methyl ether,NaN,ECHA - ED_pendiente,sin info,NaN,0,1634-04-4,216-653-1,methyl tertiary butyl ether / methyl tert-buty...,Agente oxigenante en la gasolina con olor dete...,1,NaN,0.0,NaN,EDCs DataBank/Cosing
102,Deltamethrin,NaN,Edlist_2,sin info,List II,1,52918-63-5,258-256-6,deltamethrin / decamethrin / deltamethrine/DEL...,Insecticida sintético basado en piretrinas nat...,1,NaN,0.0,NaN,EDCs DataBank/Cosing


In [34]:
# Separamos por / 
disruptores_final['nombre_etiqueta']= disruptores_final['nombre_etiqueta'].str.split(r'[/]')

In [35]:
# Descomponemos fila por cada nombre
disruptores_final = disruptores_final.explode('nombre_etiqueta')

In [36]:
disruptores_final.iloc[69:75, 6:9]

,CAS Number,EC Number,nombre_etiqueta
67,1634-04-4,216-653-1,tert-butyl methyl ether
67,1634-04-4,216-653-1,methyl t-butyl ether
67,1634-04-4,216-653-1,MTBE
67,1634-04-4,216-653-1,T-BUTYL METHYL ETHER
68,1675-54-3,216-823-5,ISOPROPYLIDENEDIPHENOL DIGLYCIDYL ETHER
69,121-00-6,204-442-7,2-TERT-BUTYLHYDROQUINONE METHYL ETHER


In [37]:
disruptores_final.iloc[108:112, 6:9]

,CAS Number,EC Number,nombre_etiqueta
102,52918-63-5,258-256-6,decamethrin
102,52918-63-5,258-256-6,deltamethrine
102,52918-63-5,258-256-6,DELTA-METHRIN
103,540-97-6,208-762-8,CYCLOHEXASILOXANE


Para análisis posteriores es importante que las variables Fuente_original y Anexo_cosing muestren más información:

In [38]:
disruptores_final["Fuente_original"].value_counts()

Fuente_original
Edlist_1                71
Edlist_2                50
ECHA - ED_pendiente     32
Edlist_3                 9
ECHA - ED_Humans&Env     1
ECHA - ED_Env            1
Name: count, dtype: int64

In [39]:
# Diccionario de mapeo
map_fuente = {
    "Edlist_1": "EDlist_1 (confirmado UE)",
    "Edlist_2": "EDlist_2 (evaluando)",
    "Edlist_3": "EDlist_3 (confirmado país miembro, pero no UE)",
    "ECHA - ED_pendiente": "ECHA (evaluando o pendiente)",
    "ECHA - ED_Env": "ECHA (efectos ambiente)",
    "ECHA - ED_Humans&Env": "ECHA (efectos salud y ambiente)"
}

# Aplicar el mapeo en la columna Fuente_original
disruptores_final["Fuente_original"] = disruptores_final["Fuente_original"].replace(map_fuente)

In [40]:
disruptores_final["Anexo_cosIng"].value_counts()

Anexo_cosIng
sin info    111
Anexo_2      19
Anexo_5      11
Anexo_3      10
Anexo_6       7
Anexo_4       6
Name: count, dtype: int64

In [41]:
map_cosing = {
    "Anexo_2": "Anexo II (Ingrediente Prohibido)",
    "Anexo_3": "Anexo III (Ingrediente Restringido)",
    "Anexo_4": "Anexo IV (Colorante Permitido)",
    "Anexo_5": "Anexo V (Conservante Permitido)",
    "Anexo_6": "Anexo VI (Filtro UV Permitido)",
    "sin info": "Sin información"
}

disruptores_final["Anexo_cosIng"] = disruptores_final["Anexo_cosIng"].replace(map_cosing)

In [42]:
disruptores_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 164 entries, 0 to 155
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name_Edlist_Echa      164 non-null    object 
 1   Name_Chemical_Cosing  52 non-null     object 
 2   Fuente_original       164 non-null    object 
 3   Anexo_cosIng          164 non-null    object 
 4   Appears_on_lists      130 non-null    object 
 5   Health_effects        164 non-null    int64  
 6   CAS Number            164 non-null    string 
 7   EC Number             164 non-null    string 
 8   nombre_etiqueta       164 non-null    object 
 9   uso                   151 non-null    object 
 10  productos_cosmeticos  164 non-null    int64  
 11  imagen_nombre         63 non-null     object 
 12  imagen_cantidad       162 non-null    float64
 13  texto                 63 non-null     object 
 14  fuente_manual         112 non-null    object 
dtypes: float64(1), int64(2), obj

In [43]:
n_dis_inicial=disruptores_provisional.shape[0]
n_alias = disruptores_final["nombre_etiqueta"].nunique()
n_cas = disruptores_final["CAS Number"].nunique()
n_ec = disruptores_final["EC Number"].nunique()
n_name_edlist = disruptores_final["Name_Edlist_Echa"].nunique()
n_texto = disruptores_final["texto"].nunique()

print(f"Número de Disruptores iniciales antes de incluir nombre de etiqueta: {n_dis_inicial}")
print(f"Alias totales conseguidos (filas de nombre_etiqueta): {n_alias}")
print(f"CAS totales (registros CAS Number único): {n_cas}")
print(f"EC totales (registros EC Number único): {n_ec}")
print(f"Disruptores con nombre INCI (registros Name_Edlist_Echa único): {n_name_edlist}")
print(f"Textos totales (registros texto único): {n_texto}")

Número de Disruptores iniciales antes de incluir nombre de etiqueta: 257
Alias totales conseguidos (filas de nombre_etiqueta): 164
CAS totales (registros CAS Number único): 79
EC totales (registros EC Number único): 78
Disruptores con nombre INCI (registros Name_Edlist_Echa único): 77
Textos totales (registros texto único): 63


Hay menos **Name_Edlist_Echa** que **CAS y EC** porque hay ingredientes que tiene más de un CAS y EC por ejemplo:

In [44]:
disruptores_final[disruptores_final["Name_Edlist_Echa"] == "Cyclomethicone"]

,Name_Edlist_Echa,Name_Chemical_Cosing,Fuente_original,Anexo_cosIng,Appears_on_lists,Health_effects,CAS Number,EC Number,nombre_etiqueta,uso,productos_cosmeticos,imagen_nombre,imagen_cantidad,texto,fuente_manual
103,Cyclomethicone,NaN,EDlist_2 (evaluando),Sin información,List II,1,540-97-6,208-762-8,CYCLOHEXASILOXANE,ACONDICIONADOR DE LA PIEL - EMOLIENTE\nACONDIC...,1,etiqueta_123.jpg/etiqueta_124.jpg,2.0,"Cyclohexasiloxane, Water, Titanium Dioxide, Di...",Cosing
104,Cyclomethicone,NaN,EDlist_2 (evaluando),Sin información,List II,1,540-97-6,208-762-8,dodecamethylcyclohexasiloxane,ACONDICIONADOR DE LA PIEL - EMOLIENTE\nACONDIC...,1,NaN,0.0,NaN,Cosing
105,Cyclomethicone,NaN,EDlist_2 (evaluando),Sin información,List II,1,540-97-6,208-762-8,Cyclomethicone 6,ACONDICIONADOR DE LA PIEL - EMOLIENTE\nACONDIC...,1,NaN,0.0,NaN,Cosing
119,Cyclomethicone,NaN,EDlist_2 (evaluando),Sin información,List II,1,69430-24-6,209-136-7,CYCLOMETHICONE,ACONDICIONADOR DE LA PIEL - EMOLIENTE\nACONDIC...,1,etiqueta_084.jpg/etiqueta_085.jpg/etiqueta_086...,3.0,"Ingredients overview\nCyclomethicone, Titanium...",Cosing


### 6.1. Conclusión
**Hemos obtenido un total de 77 de 257 disruptores con su nombre INCI, es decir un 30% del total de disruptores, el resto, pueden ser sustancias usadas en productos de limpieza, pesticidas, herbicidas, etc.., como se ha podido observar en la importación de la busqueda manual, y de los restantes no sabemos qué tipo de sustancias son.**

## 7. Exportamos la lista de diruptores final
Se exporta lista para usarla en el nlp. Tenemos en total 161 nombres de etiquetas

In [45]:
# Reiniciamos ínfice
disruptores_final.reset_index(drop=True, inplace=True)

In [46]:
disruptores_final.to_parquet("../../data/processed/notebooks/disruptores_final.parquet", index=False)

In [47]:
disruptores_final.to_excel("../../data/processed/notebooks/disruptores_final.xlsx", index=False)